# Data

In [1]:
from nibabel import load
import pandas as pd 
from pathlib import Path
from dpipe.im.visualize import slice3d
#path = '/home/aslan/Documents/self_study/hse2020_projects/training_data/survival_info.csv'
#fl = pd.read_csv(path)
folder = Path('/home/aubingazhibov/training_data/')
i = 5
img_flair = load(folder / f'BraTS20_Training_00{i}' / f'BraTS20_Training_00{i}_flair.nii.gz')
img_t1 = load(folder / f'BraTS20_Training_00{i}' / f'BraTS20_Training_00{i}_t1.nii.gz')
img_t2 = load(folder / f'BraTS20_Training_00{i}' / f'BraTS20_Training_00{i}_t2.nii.gz')
img_t1c = load(folder / f'BraTS20_Training_00{i}' / f'BraTS20_Training_00{i}_t1ce.nii.gz')
mask = load(folder / f'BraTS20_Training_00{i}' / f'BraTS20_Training_00{i}_seg.nii.gz')
pixel_array_flair = img_flair.get_fdata()
pixel_array_t1 = img_t1.get_fdata()
pixel_array_t2 = img_t2.get_fdata()
pixel_array_t1c = img_t1c.get_fdata()
pixel_array_mask = mask.get_fdata()

In [3]:
pixel_array_flair.shape

(240, 240, 155)

In [11]:
slice3d(pixel_array_flair, pixel_array_t1,
        pixel_array_t2, pixel_array_t1c, 
        pixel_array_mask,
        max_columns=5)

interactive(children=(IntSlider(value=0, continuous_update=False, description='idx', max=154), Output()), _dom…

In [3]:
slice3d(pixel_array_t1, pixel_array_t2,

        max_columns=2)

interactive(children=(IntSlider(value=0, continuous_update=False, description='idx', max=154), Output()), _dom…

In [4]:
slice3d(pixel_array_mask)

interactive(children=(IntSlider(value=0, continuous_update=False, description='idx', max=154), Output()), _dom…

# Extract the data(5 types: mask, t1, t1ce,t2, flair)

In [ ]:
import numpy as np
from tqdm import tqdm
import nibabel
from pathlib import Path

data_folder = Path('/home/aubingazhibov/training_data/')
target_folder = Path('/home/aubingazhibov/brats_slices_final/')

for subdir in tqdm(data_folder.glob('*')):
    file = subdir.stem
      
    if file != 'name_mapping' and file != 'survival_info':

 
      img_flair = nibabel.load(subdir / f'{file}_flair.nii.gz').get_fdata()
      img_t1 = nibabel.load(subdir / f'{file}_t1.nii.gz').get_fdata()
      img_t1ce = nibabel.load(subdir / f'{file}_t1ce.nii.gz').get_fdata()
      img_t2 = nibabel.load(subdir / f'{file}_t2.nii.gz').get_fdata()
      mask = nibabel.load(subdir / f'{file}_seg.nii.gz').get_fdata()
      nonzero = np.nonzero(np.sum(img_flair, (0, 1)))[0]
      subfolder = target_folder / file
      subfolder.mkdir()
      for i in nonzero:        
          np.save(subfolder / f'{i}_flair.npy', img_flair[..., i].astype(np.float32), )
          np.save(subfolder / f'{i}_t1.npy', img_t1[..., i].astype(np.float32), )
          np.save(subfolder / f'{i}_t1ce.npy', img_t1ce[..., i].astype(np.float32), )
          np.save(subfolder / f'{i}_t2.npy', img_t2[..., i].astype(np.float32), )
          np.save(subfolder / f'{i}_mask.npy', mask[..., i].astype(bool), )


In [2]:
import numpy as np
from tqdm import tqdm
import nibabel
from pathlib import Path

data_folder = Path('/home/aubingazhibov/training_data/')
target_folder = Path('/home/aubingazhibov/brats_slices/')

for subdir in tqdm(data_folder.glob('*')):
    file = subdir.stem
      
    if file != 'name_mapping' and file != 'survival_info':

 
      img_flair = nibabel.load(subdir / f'{file}_flair.nii.gz').get_fdata()
      mask = nibabel.load(subdir / f'{file}_seg.nii.gz').get_fdata()
      nonzero = np.nonzero(np.sum(img_flair, (0, 1)))[0]
      subfolder = target_folder / file
      subfolder.mkdir()
      for i in nonzero:        
          np.save(subfolder / f'{i}.npy', img_flair[..., i].astype(np.float32), )
          np.save(subfolder / f'{i}_mask.npy', mask[..., i].astype(bool), )


371it [10:43,  1.73s/it]


In [7]:
import os
import pandas as pd
from dpipe.io import load_numpy
data_folder = Path('/home/aubingazhibov/brats_slices/')
df = []
cnt = 0
for path, _, files in tqdm(os.walk(data_folder)):
    for file in files:
        subject_id = path.split('/')[-1].split('_')[-1]
        slice_id = file.split('.')[0].split('_')[0]
       # type_img = file.split('_')[1].split('.')[0]
        sample_id = f"{subject_id}_{slice_id}" # SubjectID_SliceIndex
        is_mask = 'mask' in file
        if is_mask:
            mask = load_numpy(Path(path) / file, allow_pickle=True)
            is_nonzero_mask =  np.any(mask)
        else:
            is_nonzero_mask = np.nan
        
        df.append([Path(Path(path).stem) / file, sample_id, is_mask, subject_id, is_nonzero_mask])
    
    
df = pd.DataFrame(df, columns = ['relative_path','sample_id', 'is_mask', 'subject_id', 'is_nonzero_mask'])
print(df.is_nonzero_mask.value_counts())

df.to_csv(data_folder / 'meta.csv')

370it [01:01,  6.00it/s]


False    26422
True     24422
Name: is_nonzero_mask, dtype: int64


# Creating CSV

In [ ]:
data_folder = Path('/home/aubingazhibov/brats_slices_final/')
df = []
cnt = 0
for path, _, files in tqdm(os.walk(data_folder)):
    for file in files:
        subject_id = path.split('/')[-1].split('_')[-1]
        slice_id = file.split('.')[0].split('_')[0]
        type_img = file.split('_')[1].split('.')[0]
        sample_id = f"{subject_id}_{slice_id}" # SubjectID_SliceIndex
        is_mask = 'mask' in file
        if is_mask:
            mask = load_numpy(Path(path) / file, allow_pickle=True)
            is_nonzero_mask =  np.any(mask)
        else:
            is_nonzero_mask = np.nan
        
        df.append([Path(Path(path).stem) / file, type_img, sample_id, is_mask, subject_id, is_nonzero_mask])
    
    
df = pd.DataFrame(df, columns = ['relative_path','img_type', 'sample_id', 'is_mask', 'subject_id', 'is_nonzero_mask'])
print(df.is_nonzero_mask.value_counts())

df.to_csv(data_folder / 'meta.csv')

In [ ]:
data_folder = Path('/home/aubingazhibov/brats_slices_final')
df = pd.read_csv(data_folder / 'meta.csv', index_col=0)
df.sample(10)